## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Ushuaia,AR,2022-05-07 21:59:35,-54.8000,-68.3000,49.66,46,20,21.85,few clouds
1,1,Punta Arenas,CL,2022-05-07 21:59:36,-53.1500,-70.9167,48.31,71,20,23.02,few clouds
2,2,Avarua,CK,2022-05-07 21:59:36,-21.2078,-159.7750,80.53,74,100,5.30,overcast clouds
3,3,Hobart,AU,2022-05-07 21:58:07,-42.8794,147.3294,53.46,82,75,12.66,broken clouds
4,4,Lasa,IT,2022-05-07 21:59:37,46.6166,10.7002,50.36,95,100,2.75,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip?  70
What is the maximum temperature you would like for your trip?  90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                       (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
2,2,Avarua,CK,2022-05-07 21:59:36,-21.2078,-159.7750,80.53,74,100,5.30,overcast clouds
9,9,Saint-Philippe,RE,2022-05-07 21:53:11,-21.3585,55.7679,72.99,83,33,10.87,light rain
10,10,Puerto Ayora,EC,2022-05-07 21:50:13,-0.7393,-90.3518,80.56,80,33,9.40,scattered clouds
11,11,Georgetown,MY,2022-05-07 21:59:40,5.4112,100.3354,84.13,92,20,11.50,few clouds
12,12,Mbaiki,CF,2022-05-07 21:59:41,3.8678,17.9892,72.05,84,80,1.01,broken clouds
14,14,Arraial Do Cabo,BR,2022-05-07 21:50:27,-22.9661,-42.0278,73.36,78,40,9.22,scattered clouds
17,17,Rikitea,PF,2022-05-07 21:59:43,-23.1203,-134.9692,77.02,84,53,14.29,light rain
19,19,Vaini,TO,2022-05-07 21:53:06,-21.2000,-175.2000,77.16,83,75,12.66,broken clouds
28,28,Camopi,GF,2022-05-07 21:57:56,3.1655,-52.3158,74.17,99,88,2.95,overcast clouds
31,31,Glenwood Springs,US,2022-05-07 21:59:50,39.5505,-107.3248,79.72,32,75,17.27,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                266
City                   266
Country                266
Date                   266
Lat                    266
Lng                    266
Max Temp               266
Humidity               266
Cloudiness             266
Wind Speed             266
Weather Description    266
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_df = preferred_cities_df.dropna(axis=0, how='any')
print(len(clean_preferred_df))

266


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Avarua,CK,80.53,overcast clouds,-21.2078,-159.7750,
9,Saint-Philippe,RE,72.99,light rain,-21.3585,55.7679,
10,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,
11,Georgetown,MY,84.13,few clouds,5.4112,100.3354,
12,Mbaiki,CF,72.05,broken clouds,3.8678,17.9892,
14,Arraial Do Cabo,BR,73.36,scattered clouds,-22.9661,-42.0278,
17,Rikitea,PF,77.02,light rain,-23.1203,-134.9692,
19,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,
28,Camopi,GF,74.17,overcast clouds,3.1655,-52.3158,
31,Glenwood Springs,US,79.72,broken clouds,39.5505,-107.3248,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location']=f'{lat},{lng}'

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
print(len(hotel_df))
hotel_df.head(10)

266


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Avarua,CK,80.53,overcast clouds,-21.2078,-159.7750,Paradise Inn
9,Saint-Philippe,RE,72.99,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
10,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
11,Georgetown,MY,84.13,few clouds,5.4112,100.3354,Cititel Penang
12,Mbaiki,CF,72.05,broken clouds,3.8678,17.9892,Motel MBM
14,Arraial Do Cabo,BR,73.36,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
17,Rikitea,PF,77.02,light rain,-23.1203,-134.9692,People ThankYou
19,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
28,Camopi,GF,74.17,overcast clouds,3.1655,-52.3158,La pousada
31,Glenwood Springs,US,79.72,broken clouds,39.5505,-107.3248,Glenwood Springs Cedar Lodge


In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0, how='any')
print(len(clean_hotel_df))
clean_hotel_df.head(10)

266


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Avarua,CK,80.53,overcast clouds,-21.2078,-159.7750,Paradise Inn
9,Saint-Philippe,RE,72.99,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
10,Puerto Ayora,EC,80.56,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
11,Georgetown,MY,84.13,few clouds,5.4112,100.3354,Cititel Penang
12,Mbaiki,CF,72.05,broken clouds,3.8678,17.9892,Motel MBM
14,Arraial Do Cabo,BR,73.36,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
17,Rikitea,PF,77.02,light rain,-23.1203,-134.9692,People ThankYou
19,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
28,Camopi,GF,74.17,overcast clouds,3.1655,-52.3158,La pousada
31,Glenwood Springs,US,79.72,broken clouds,39.5505,-107.3248,Glenwood Springs Cedar Lodge


In [31]:
# 8a. Create the output File (CSV)
output_data_file = 'Vacation_Search/WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp}F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[['Lat','Lng']]
max_temp = clean_hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                 max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))